## 1. Baixe o conjunto de dados Iris (https://www.kaggle.com/uciml/iris):


In [524]:
import pandas as pd
import numpy as np

In [525]:
# reading iris dataset
iris = pd.read_csv("dataset/iris.csv", index_col=0)
print(iris.head())
print(iris.shape)

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
Id                                                                       
1             5.1           3.5            1.4           0.2  Iris-setosa
2             4.9           3.0            1.4           0.2  Iris-setosa
3             4.7           3.2            1.3           0.2  Iris-setosa
4             4.6           3.1            1.5           0.2  Iris-setosa
5             5.0           3.6            1.4           0.2  Iris-setosa
(150, 5)


## 2. Exiba a quantidade de amostras existente em cada classe.

In [526]:
# separated by species
type_setosa = iris[iris["Species"] == "Iris-setosa"]
type_virginica = iris[iris["Species"] == "Iris-virginica"]
type_versicolor = iris[iris["Species"] == "Iris-versicolor"]
# number of observations
print("Iris-setosa: " + str(type_setosa.shape[0]) + " amostras")
print("Iris-virginica: " + str(type_virginica.shape[0]) + " amostras")
print("Iris-versicolor: " + str(type_versicolor.shape[0]) + " amostras")

Iris-setosa: 50 amostras
Iris-virginica: 50 amostras
Iris-versicolor: 50 amostras


## 3. Separe aleatoriamente 80% dos dados para treino (conjunto de treino) e 20% dos dados para teste (conjunto de teste). Para esta separação, use somente recursos do Python, Numpy ou Pandas. Não use o Scikit Learn para isso.

In [527]:
# list to separete train data (80% - 120 observations) and test data (20% - 30 observations)
a_test = np.array([False]*50)
index = 0
while index < 10 :
    num = np.random.randint(0,50, random_state=1) # create 10 random values to set True on the a_train indexes
    if a_test[num] == False :
        a_test[num] = True
        index += 1

# create a test dataframe with 30 observations
test_data = pd.concat([type_setosa[a_test], type_versicolor[a_test], type_virginica[a_test]])

# invert all values of a_train to crate the test dataframe
a_train = np.invert(a_test)

# create a train dataframe with 120 observations
train_data = pd.concat([type_setosa[a_train], type_versicolor[a_train], type_virginica[a_train]])

print(train_data)
print(train_data.shape)
print(test_data)
print(test_data.shape)

TypeError: randint() got an unexpected keyword argument 'random_state'

In [450]:
# randomize the test and train data
test_data = test_data.sample(frac=1, random_state=42)
train_data = train_data.sample(frac=1, random_state=42)
#print(test_data)
#print(train_data)

## 4. Usando somente o conjunto de treino, mostre as médias dos valores de cada feature para cada classe.

In [451]:
train_data.groupby("Species").mean()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,
Iris-setosa,5.0250,3.4375,1.4575,0.2350
Iris-versicolor,5.9850,2.7875,4.2950,1.3525
Iris-virginica,6.5975,3.0225,5.5675,2.0500


## 5. Crie um algoritmo seu (sem usar aprendizado de máquina) para baseado nas médias das features calculadas no item 4, ou mesmo usando outras ideias suas, classificar corretamente as amostras do conjunto de teste. Aperfeçoe o seu algoritmo de forma a obter melhores resultados.

In [452]:
# Analyze correlation between the features
train_data.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,120.000000,120.000000,120.000000,120.000000
mean,5.869167,3.082500,3.773333,1.212500
std,0.829072,0.433989,1.776681,0.777149
min,4.400000,2.000000,1.000000,0.100000
25%,5.200000,2.800000,1.500000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.400000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [453]:
def personal_classifier(df_test:pd.DataFrame):
    '''
    Parameters: df_test = pandas.DataFrame (test dataset).
    Return: numpy.Array = ["Iris_setosa", "Iris_versicolor" ou "Iris_virginica"] (classification type).
    '''

    classification = ""

    train_petal_info = train_data.groupby("Species").mean()
    print(train_petal_info)
    train_petal_info = train_petal_info[["PetalLengthCm", "PetalWidthCm"]]
    np_train_petal_lenght = np.array(train_petal_info.iloc[:, 0])
    np_train_petal_width = np.array(train_petal_info.iloc[:, 1])
    #print(np_train_petal_lenght)
    #print(train_petal_info)

    test_petal_info = df_test[["PetalLengthCm", "PetalWidthCm"]]
    np_test_petal_lenght = np.array(test_petal_info.iloc[:, 0])
    np_test_petal_width = np.array(test_petal_info.iloc[:, 1])
    #print(np_test_petal_lenght)

    l_train = []

    # Setosa test
    train = np_train_petal_lenght[0]
    for test in np_test_petal_lenght :
        l_train.append([abs(train - test), "Iris-setosa"])
    
    # Versicolor test
    index = 0
    train = np_train_petal_lenght[1]
    for test in np_test_petal_lenght :
        if (abs(train - test) < l_train[index][0]) :
            l_train[index] = [abs(train - test), "Iris-versicolor"]
        index += 1
    
    # Virginica test
    index = 0
    train = np_train_petal_lenght[2]
    for test in np_test_petal_lenght :
        if (abs(train - test) < l_train[index][0]) :
            l_train[index] = [abs(train - test), "Iris-virginica"]
        index += 1
    
    np_l_train = np.array(l_train)
    return np_l_train[:,1]

In [454]:
predicted_values = personal_classifier(test_data)
print(predicted_values)

                 SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
Species                                                                  
Iris-setosa             5.0250        3.4375         1.4575        0.2350
Iris-versicolor         5.9850        2.7875         4.2950        1.3525
Iris-virginica          6.5975        3.0225         5.5675        2.0500
['Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica'
 'Iris-setosa' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa']


## 6. Quanto por cento no conjunto de teste você conseguiu acertar corretamente o rótulo (classe) no seu melhor algoritmo, ou seja, qual a acurácia?

In [455]:
def personal_evaluation(real_values, predicted_values) :
    count = 0
    for index in range(30) :
        if real_values[index] == predicted_values[index] :
            count += 1
        #print(real_values[index], predicted_values[index])
            
    return round(count*100 / real_values.shape[0], 2)
    

real_values = np.array(test_data["Species"])
evaluation = personal_evaluation(real_values, predicted_values)
print(str(evaluation) + " %")

96.67 %
